In [3]:
# 0. Google Colab GPU kontrolü
# =============================
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Cihaz:", device)


Cihaz: cuda


In [4]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np


In [5]:
# 2. Veri Seti Yükleme
# =============================
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Label Encoding
emotion_encoder = LabelEncoder()
train_df["emotion_label"] = emotion_encoder.fit_transform(train_df["emotion"])


In [6]:
# 3. Dataset Class
# =============================
class MentalHealthDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }


In [7]:
# 4. Tokenizer ve Dataset
# =============================
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
train_dataset = MentalHealthDataset(train_df["text"].values, train_df["emotion_label"].values, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [8]:
# 5. Model
# =============================
model = AutoModelForSequenceClassification.from_pretrained(
    "dbmdz/bert-base-turkish-cased",
    num_labels=len(emotion_encoder.classes_)
)
model.to(device)

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [9]:
# 6. Optimizer ve Loss
# =============================
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [11]:
# 7. Eğitim Döngüsü
# =============================
epochs = 3

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} Loss: {avg_loss:.4f}")


Epoch 1: 100%|██████████| 313/313 [00:55<00:00,  5.68it/s]


Epoch 1 Loss: 0.0943


Epoch 2: 100%|██████████| 313/313 [00:58<00:00,  5.37it/s]


Epoch 2 Loss: 0.0056


Epoch 3: 100%|██████████| 313/313 [00:58<00:00,  5.38it/s]

Epoch 3 Loss: 0.0027


In [12]:
# 8. Model Kaydetme
# =============================
model.save_pretrained("./emotion_model_turkish_quick")
tokenizer.save_pretrained("./emotion_model_turkish_quick")

print("✅ Hızlı eğitim tamamlandı ve model kaydedildi!")

✅ Hızlı eğitim tamamlandı ve model kaydedildi!


In [13]:
# 9. Modeli Yükleme (Eğitim sonrası)
# =============================
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_path = "./emotion_model_turkish_quick"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()  # Tahmin için eval moduna al

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [14]:
# 10. Tahmin Fonksiyonu
# =============================
def predict_emotion(text_list):
    encodings = tokenizer(
        text_list,
        padding=True,
        truncation=True,
        max_length=64,
        return_tensors="pt"
    )
    input_ids = encodings["input_ids"].to(device)
    attention_mask = encodings["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()

    # LabelEncoder kullanarak sınıf isimlerini geri dön
    return emotion_encoder.inverse_transform(preds)


In [15]:
# =============================
# 11. Test Verisi ile Tahmin
# =============================
texts = test_df["text"].tolist()
predictions = predict_emotion(texts)

# Sonuçları DataFrame olarak ekle
test_df["predicted_emotion"] = predictions
test_df.head(10)

,text,emotion,stress_level,risk,predicted_emotion
0,Bu durumdan tiksindim.,iğrenme,yüksek,hayır,iğrenme
1,"Ne olduğunu anlayamadım, gerçekten şaşırdım.",şaşkınlık,orta,hayır,şaşkınlık
2,Bugün gerçekten çok mutluyum.,mutluluk,düşük,hayır,mutluluk
3,Gerçekten mide bulandırıcı.,iğrenme,yüksek,hayır,iğrenme
4,Her şey üstüme geliyor gibi.,üzüntü,düşük,hayır,üzüntü
5,Bugün çok öfkeliyim.,öfke,orta,hayır,öfke
6,Hiçbir şey yapmak istemiyorum.,üzüntü,yüksek,hayır,üzüntü
7,Gerçekten böyle olacağını hiç düşünmemiştim.,şaşkınlık,düşük,hayır,şaşkınlık
8,İğrenç bir davranıştı.,iğrenme,orta,hayır,iğrenme
9,Bugün gerçekten çok mutluyum.,mutluluk,orta,hayır,mutluluk


In [16]:
# 12. Örnek Cümle ile Tahmin
# =============================
example_text = ["Bugün çok stresliyim ve hiç motive olamıyorum."]  # Buraya istediğin cümleyi yazabilirsin

predicted_emotion = predict_emotion(example_text)

print(f"Örnek Cümle: {example_text[0]}")
print(f"Tahmin Edilen Duygu: {predicted_emotion[0]}")

Örnek Cümle: Bugün çok stresliyim ve hiç motive olamıyorum.
Tahmin Edilen Duygu: üzüntü


In [17]:
# 12. Örnek Cümle ile Tahmin
# =============================
example_text = ["Bugün harika hissediyorum."]  # Buraya istediğin cümleyi yazabilirsin

predicted_emotion = predict_emotion(example_text)

print(f"Örnek Cümle: {example_text[0]}")
print(f"Tahmin Edilen Duygu: {predicted_emotion[0]}")

Örnek Cümle: Bugün harika hissediyorum.
Tahmin Edilen Duygu: mutluluk


In [18]:
# 12. Örnek Cümle ile Tahmin
# =============================
example_text = ["Bunu demesini beklemiyordum."]  # Buraya istediğin cümleyi yazabilirsin

predicted_emotion = predict_emotion(example_text)

print(f"Örnek Cümle: {example_text[0]}")
print(f"Tahmin Edilen Duygu: {predicted_emotion[0]}")

Örnek Cümle: Bunu demesini beklemiyordum.
Tahmin Edilen Duygu: şaşkınlık
